In [1]:
#import from regular python

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb

import os
import re
import getpass

user = getpass.getuser()

if user == "sidsel":
    cvr_Path = "/home/"+user+"/workspace/python/DABAI/ReadData"

elif user == "svanhmic":
    cvr_Path = "/home/"+user+"/workspace/DABAI/ReadData" # TODB alter path!



In [88]:
#imports from Pyspark

from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans

sc.addPyFile(cvr_Path+"/ConvertAllToVecToMl.py")
from ConvertAllToVecToMl import ConvertAllToVecToMl

In [89]:
data_Pdf = sb.load_dataset("flights") # naive data set from seaborn
#data_Pdf

In [90]:
#create spark data frame
flightDf = sqlContext.createDataFrame(data_Pdf)

# Convering month to long
import datetime as dt
func = F.udf(lambda x: dt.datetime.strptime(x, '%b').month, LongType())
flightInfoDf = flightDf.select('*').withColumn('month', func(F.substring(str='month',pos=0,len=3)))

+---------+
|    month|
+---------+
|     July|
| November|
| February|
|  January|
|    March|
|  October|
|      May|
|   August|
|    April|
|     June|
| December|
|September|
+---------+



In [97]:
#create the spark ml pipeline

vectorAssembler = VectorAssembler(inputCols=flightDf.columns,outputCol="features")
convertion = ConvertAllToVecToMl(inputCol=vectorAssembler.getOutputCol(), outputCol=vectorAssembler.getOutputCol())
clusters = KMeans(featuresCol=vectorAssembler.getOutputCol(), k=2, maxIter=10, initMode="random")

pipeline = Pipeline(stages=[vectorAssembler, convertion, clusters])


In [116]:
#execute the pipeline

model = pipeline.fit(flightInfoDf)
prediction = model.transform(flightInfoDf)


In [115]:
#return the data to Pandas for visualisation

prediction.toPandas()


,year,month,passengers,features,prediction
0,1949,1,112,"[1949.0, 1.0, 112.0]",0
1,1949,2,118,"[1949.0, 2.0, 118.0]",0
2,1949,3,132,"[1949.0, 3.0, 132.0]",0
3,1949,4,129,"[1949.0, 4.0, 129.0]",0
4,1949,5,121,"[1949.0, 5.0, 121.0]",0
5,1949,6,135,"[1949.0, 6.0, 135.0]",0
6,1949,7,148,"[1949.0, 7.0, 148.0]",0
7,1949,8,148,"[1949.0, 8.0, 148.0]",0
8,1949,9,136,"[1949.0, 9.0, 136.0]",0
9,1949,10,119,"[1949.0, 10.0, 119.0]",0
